In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#from tslearn.preprocessing import TimeSeriesScalerMeanVariance
#from tslearn.clustering import TimeSeriesKMeans
#from joblib import Parallel, delayed
#from tslearn.clustering import TimeSeriesKMeans
#from sklearn.preprocessing import MinMaxScaler
#import pickle

In [2]:
path_products = './datasets/tb_productos.txt'
path_stocks = './datasets/tb_stocks.txt'
path_sells = './datasets/sell-in.txt'
path_products_to_predict = './datasets/product_id_apredecir201912.txt'

# 📁 Crear carpeta datasets (si no existe)
!mkdir -p datasets

# 📥 Descargar archivos desde la URL pública
!wget -q https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/tb_productos.txt -O ./datasets/tb_productos.txt
!wget -q https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/tb_stocks.txt -O ./datasets/tb_stocks.txt
!wget -q https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/sell-in.txt.gz -O ./datasets/sell-in.txt.gz
!wget -q https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt -O ./datasets/product_id_apredecir201912.txt

# 🗜️ Descomprimir el archivo sell-in.txt.gz
import gzip
import shutil

with gzip.open('./datasets/sell-in.txt.gz', 'rb') as f_in:
    with open('./datasets/sell-in.txt', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print("✅ Archivos descargados y preparados correctamente.")

Ya existe el subdirectorio o el archivo -p.
Error mientras se procesaba: -p.
Ya existe el subdirectorio o el archivo datasets.
Error mientras se procesaba: datasets.
"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


✅ Archivos descargados y preparados correctamente.


In [3]:
tn_scaler_path='./scalers/scalers.pkl'
encoders_path='./encoders/encoders.pkl'
categorical_cols = ['cat1', 'cat2', 'cat3', 'brand', 'sku_size']

df_products = pd.read_csv(path_products, sep='\t', dtype={ 'product_id': str, 'sku_size': str })
df_stocks = pd.read_csv(path_stocks, sep='\t', dtype={ 'product_id': str, 'periodo': str })
df_sellin = pd.read_csv(path_sells, sep='\t', dtype={ 'product_id': str, 'customer_id': str, 'periodo': str })
df_products_to_predict = pd.read_csv(path_products_to_predict, dtype={ 'product_id': str })



In [4]:
# 🧹 3. Preprocesamiento
# Convertir periodo a datetime
df_sellin['timestamp'] = pd.to_datetime(df_sellin['periodo'], format='%Y%m')

In [5]:
# 📄 Leer lista de productos a predecir
with open("./datasets/product_id_apredecir201912.txt", "r") as f:
    product_ids = [int(line.strip()) for line in f if line.strip().isdigit()]

# prompt: como aplico esto, a mi script
# # 📄 Leer lista de productos a predecir
# with open("product_id_apredecir201912.TXT", "r") as f:
#     product_ids = [int(line.strip()) for line in f if line.strip().isdigit()]

# 📄 Leer lista de productos a predecir
with open(path_products_to_predict, "r") as f:
    product_ids_to_predict = [int(line.strip()) for line in f if line.strip().isdigit()]

# Filtrar df_sellin para incluir solo los product_ids_to_predict
df_sellin_filtered = df_sellin[df_sellin['product_id'].astype(int).isin(product_ids_to_predict)].copy()


In [6]:
df_products

,cat1,cat2,cat3,brand,sku_size,product_id,descripcion
0,FOODS,ADEREZOS,Aji Picante,NATURA,240,20609,Salsa Aji Picante
1,FOODS,ADEREZOS,Barbacoa,NATURA,250,20266,Salsa Barbacoa
2,FOODS,ADEREZOS,Barbacoa,NATURA,400,20325,Salsa Barbacoa
3,FOODS,ADEREZOS,Barbacoa,NATURA,500,20503,Salsa Barbacoa
4,FOODS,ADEREZOS,Chimichurri,NATURA,350,20797,Chimichurri
...,...,...,...,...,...,...,...
1246,REF,TE,Frutas,TWININGS,20,21271,Frutas
1247,REF,TE,Hierbas,TWININGS,20,21202,Manzanilla
1248,REF,TE,Hierbas,TWININGS,20,21218,Menta
1249,REF,TE,Verde,TWININGS,20,21192,Verde


In [7]:
df_products_to_predict

,product_id
0,20001
1,20002
2,20003
3,20004
4,20005
...,...
775,21263
776,21265
777,21266
778,21267


In [8]:
df_sellin.columns

Index(['periodo', 'customer_id', 'product_id', 'plan_precios_cuidados',
       'cust_request_qty', 'cust_request_tn', 'tn', 'timestamp'],
      dtype='object')

In [9]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Asegurar que product_id es str en todos
df_sellin['product_id'] = df_sellin['product_id'].astype(str)
df_products['product_id'] = df_products['product_id'].astype(str)
df_products_to_predict['product_id'] = df_products_to_predict['product_id'].astype(str)

# Paso 1: Agregamos tn por product_id y periodo
df_agg = df_sellin.groupby(['product_id', 'periodo'], as_index=False)['tn'].sum()
df_agg = df_agg.sort_values(['product_id', 'periodo'])
df_agg['periodo'] = df_agg['periodo'].astype(int)


In [10]:
df_agg

,product_id,periodo,tn
0,20001,201701,934.77222
1,20001,201702,798.01620
2,20001,201703,1303.35771
3,20001,201704,1069.96130
4,20001,201705,1502.20132
...,...,...,...
31238,21295,201701,0.00699
31239,21296,201708,0.00651
31240,21297,201701,0.00579
31241,21298,201708,0.00573


In [11]:
# prompt: como hago para filtrar en df_agg para los product_id que estan en df_products_to_predict

df_agg_filtered = df_agg[df_agg['product_id'].isin(df_products_to_predict['product_id'])]
df_agg_filtered

,product_id,periodo,tn
0,20001,201701,934.77222
1,20001,201702,798.01620
2,20001,201703,1303.35771
3,20001,201704,1069.96130
4,20001,201705,1502.20132
...,...,...,...
31206,21276,201908,0.01265
31207,21276,201909,0.01856
31208,21276,201910,0.02079
31209,21276,201911,0.03341


HASTA ACA LA REGRESION, PROBAMOS AUTOGULON

In [12]:
# 💬 Instalar AutoGluon si es necesario
%pip install autogluon.timeseries

from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


c:\Users\TOMAS\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
df_sellin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945818 entries, 0 to 2945817
Data columns (total 8 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   periodo                object        
 1   customer_id            object        
 2   product_id             object        
 3   plan_precios_cuidados  int64         
 4   cust_request_qty       int64         
 5   cust_request_tn        float64       
 6   tn                     float64       
 7   timestamp              datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 179.8+ MB


In [14]:
# Asegurar que los IDs sean strings
product_ids = [str(p) for p in product_ids]

# Filtro hasta fin de diciembre 2019 y productos requeridos
df_filtered = df_sellin[
    (df_sellin['timestamp'] <= '2019-10-31') &
    (df_sellin['product_id'].isin(product_ids))
]


In [15]:
df_filtered

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,timestamp
0,201701,10234,20524,0,2,0.05300,0.05300,2017-01-01
1,201701,10032,20524,0,1,0.13628,0.13628,2017-01-01
2,201701,10217,20524,0,1,0.03028,0.03028,2017-01-01
3,201701,10125,20524,0,1,0.02271,0.02271,2017-01-01
4,201701,10012,20524,0,11,1.54452,1.54452,2017-01-01
...,...,...,...,...,...,...,...,...
2814448,201910,10110,20853,0,1,0.00669,0.00669,2019-10-01
2814449,201910,10321,20853,0,1,0.00446,0.00446,2019-10-01
2814450,201910,10443,20853,0,1,0.11148,0.11148,2019-10-01
2814451,201910,10086,20853,0,2,0.02007,0.02007,2019-10-01


In [16]:
# Agregar tn por periodo, cliente y producto
df_grouped = df_filtered.groupby(['timestamp', 'customer_id', 'product_id'], as_index=False)['tn'].sum()

In [17]:
# Agregar tn total por periodo y producto
df_monthly_product = df_grouped.groupby(['timestamp', 'product_id'], as_index=False)['tn'].sum()

In [18]:
# Agregar columna 'item_id' para AutoGluon
df_monthly_product['item_id'] = df_monthly_product['product_id']

In [19]:
# ⏰ 4. Crear TimeSeriesDataFrame
ts_data = TimeSeriesDataFrame.from_data_frame(
    df_monthly_product,
    id_column='item_id',
    timestamp_column='timestamp'
)

In [20]:
# Completar valores faltantes
ts_data = ts_data.fill_missing_values()

In [21]:
ts_data

,,product_id,tn
item_id,timestamp,,
20001,2017-01-01,20001,934.77222
20002,2017-01-01,20002,550.15707
20003,2017-01-01,20003,1063.45835
20004,2017-01-01,20004,555.91614
20005,2017-01-01,20005,494.27011
...,...,...,...
21263,2019-10-01,21263,0.01552
21265,2019-10-01,21265,0.10921
21266,2019-10-01,21266,0.11831


In [32]:
# ⚙️ 5. Definir y entrenar predictor
predictor = TimeSeriesPredictor(
    prediction_length=2,
    target='tn',
    freq='MS'  # Frecuencia mensual (Month Start),
)

predictor.fit(ts_data, num_val_windows=2, time_limit=60*60)

Beginning AutoGluon training... Time limit = 3600s
AutoGluon will save models to 'c:\Users\TOMAS\Downloads\AutogluonModels\ag-20250719_185538'


=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.12.2
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          8
GPU Count:          0
Memory Avail:       1.24 GB / 7.89 GB (15.7%)
Disk Space Avail:   83.25 GB / 237.24 GB (35.1%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'MS',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'tn',
 'time_limit': 3600,
 'verbosity': 2}

train_data with frequency 'IRREG' has been resampled to frequency 'MS'.
Provided train_data has 20815 rows (NaN fraction=0.1%), 780 time series. Median time series length is 34 (min=2, max=34). 
	Removing 120 short time series from train_data. Onl

In [33]:
# 🔮 6. Generar predicción
forecast = predictor.predict(ts_data)

data with frequency 'IRREG' has been resampled to frequency 'MS'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [34]:
# Extraer predicción media y filtrar febrero 2020
forecast_mean = forecast['mean'].reset_index()
print(forecast_mean.columns)

Index(['item_id', 'timestamp', 'mean'], dtype='object')


In [35]:
# Tomar solo item_id y la predicción 'mean'
resultado = forecast['mean'].reset_index()[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']

# Filtrar solo febrero 2020
resultado = forecast['mean'].reset_index()
resultado = resultado[resultado['timestamp'] == '2019-12-01']

# Renombrar columnas
resultado = resultado[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']

In [36]:
# 💾 7. Guardar archivo
resultado.to_csv("predicciones_autogulon basico diciembre.csv", index=False)
resultado.head()

,product_id,tn
1,20001,1494.019378
3,20002,1293.728817
5,20003,957.351064
7,20004,738.793648
9,20005,690.431738


In [22]:
# Asegurarse de que timestamp sea una columna y no índice
ts_data = ts_data.reset_index()  # Si ya no lo está
df_products_renamed = df_products.rename(columns={'product_id': 'item_id'})
ts_data_enriched = ts_data.merge(df_products_renamed, on='item_id', how='left')


In [23]:
ts_data_enriched

,item_id,timestamp,product_id,tn,cat1,cat2,cat3,brand,sku_size,descripcion
0,20001,2017-01-01,20001,934.77222,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
1,20002,2017-01-01,20002,550.15707,HC,ROPA LAVADO,Liquido,LIMPIEX,3000,Maquina 1er lavado
2,20003,2017-01-01,20003,1063.45835,FOODS,ADEREZOS,Mayonesa,NATURA,475,Regular sin TACC
3,20004,2017-01-01,20004,555.91614,FOODS,ADEREZOS,Mayonesa,NATURA,240,Regular sin TACC
4,20005,2017-01-01,20005,494.27011,FOODS,ADEREZOS,Mayonesa,NATURA,120,Regular sin TACC
...,...,...,...,...,...,...,...,...,...,...
20784,21263,2019-10-01,21263,0.01552,PC,CABELLO,SHAMPOO,VICHY,250,Sabor 1
20785,21265,2019-10-01,21265,0.10921,PC,PIEL1,CUIDADO ESPECIAL,LANCOME,32,mascara facial 2
20786,21266,2019-10-01,21266,0.11831,PC,PIEL1,CUIDADO ESPECIAL,LANCOME,32,mascara facial 1
20787,21267,2019-10-01,21267,0.09676,PC,PIEL1,Cara,NIVEA,250,Limpieza Facial


In [24]:
ts_data_enriched.columns
# ['item_id', 'timestamp', 'tn', 'cat1', 'cat2', ..., 'descripcion']


Index(['item_id', 'timestamp', 'product_id', 'tn', 'cat1', 'cat2', 'cat3',
       'brand', 'sku_size', 'descripcion'],
      dtype='object')

In [25]:
# 1. Filtrar datos hasta dic-2019
ts_final_train = ts_data_enriched[ts_data_enriched['timestamp'] <= '2019-10-01']

# 2. Establecer índice jerárquico
ts_final_train = ts_final_train.set_index(['item_id', 'timestamp']).sort_index()


In [26]:
ts_final_train

product_id          tn cat1         cat2     cat3  brand  \
item_id timestamp                                                             
20001   2017-01-01      20001   934.77222   HC  ROPA LAVADO  Liquido  ARIEL   
        2017-02-01      20001   798.01620   HC  ROPA LAVADO  Liquido  ARIEL   
        2017-03-01      20001  1303.35771   HC  ROPA LAVADO  Liquido  ARIEL   
        2017-04-01      20001  1069.96130   HC  ROPA LAVADO  Liquido  ARIEL   
        2017-05-01      20001  1502.20132   HC  ROPA LAVADO  Liquido  ARIEL   
...                       ...         ...  ...          ...      ...    ...   
21276   2019-06-01      21276     0.04086   PC        PIEL1     Cara  NIVEA   
        2019-07-01      21276     0.00223   PC        PIEL1     Cara  NIVEA   
        2019-08-01      21276     0.01265   PC        PIEL1     Cara  NIVEA   
        2019-09-01      21276     0.01856   PC        PIEL1     Cara  NIVEA   
        2019-10-01      21276     0.02079   PC        PIEL1     Cara  NIVEA   

                   sku_size     descripcion  
item_id timestamp                            
20001   2017-01-01     3000          genoma  
        2017-02-01     3000          genoma  
        2017-03-01     3000          genoma  
        2017-04-01     3000          genoma  
        2017-05-01     3000          genoma  
...                     ...             ...  
21276   2019-06-01      140  reconstruccion  
        2019-07-01      140  reconstruccion  
        2019-08-01      140  reconstruccion  
        2019-09-01      140  reconstruccion  
        2019-10-01      140  reconstruccion  

[20789 rows x 8 columns]

In [42]:

# 3. Entrenar predictor
predictor_final = TimeSeriesPredictor(
    prediction_length=2,
    target='tn',
    freq='MS'
)

predictor_final.fit(
    ts_final_train,
    num_val_windows=4,
    presets='best_quality',
    time_limit=2*60*60
)



Beginning AutoGluon training... Time limit = 7200s
AutoGluon will save models to 'c:\Users\TOMAS\Downloads\AutogluonModels\ag-20250719_191925'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.12.2
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          8
GPU Count:          0
Memory Avail:       1.39 GB / 7.89 GB (17.6%)
Disk Space Avail:   82.99 GB / 237.24 GB (35.0%)
Setting presets to: best_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'MS',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 4,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'tn',
 'time_limit': 7200,
 'verbosity': 2}

train_data with frequency 'IRREG' has been resampled to frequency 'MS'.

In [43]:

# 3. Predecir ene-feb 2020
forecast_final = predictor_final.predict(ts_final_train)
forecast_df = forecast_final['mean'].reset_index()

# 4. Filtrar solo febrero 2020
forecast_dic2019 = forecast_df[forecast_df['timestamp'] == '2019-12-01']
forecast_dic2019 = forecast_dic2019.rename(columns={'mean': 'tn', 'item_id': 'product_id'})

# 5. Exportar resultados
forecast_dic2019[['product_id', 'tn']].to_csv('predicciones_diciembre2019_autogluon1.csv', index=False)

data with frequency 'IRREG' has been resampled to frequency 'MS'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [64]:
from autogluon.timeseries import TimeSeriesPredictor
import numpy as np

# 1. Preprocesamiento: casteo y transformación log1p
ts_data_transformed = ts_data_enriched.copy()
for col in ['cat1', 'cat2', 'cat3', 'brand']:
    ts_data_transformed[col] = ts_data_transformed[col].astype('category')

ts_data_transformed['tn'] = np.log1p(ts_data_transformed['tn'])

# 2. Entrenar hasta dic-2019
ts_final_train = ts_data_transformed[ts_data_transformed['timestamp'] <= '2019-10-31']
ts_final_train = ts_final_train.set_index(['item_id', 'timestamp']).sort_index()



In [65]:
# 3. Entrenar modelo avanzado
predictor_advanced = TimeSeriesPredictor(
    prediction_length=2,
    target='tn',
    freq='MS',
    eval_metric='WAPE'
)

predictor_advanced.fit(
    ts_final_train,
    hyperparameters={
        'DeepAR': {},
        'ETS': {},
        'Naive': {}
    },
    num_val_windows=5,
    time_limit=2*60*60
)

No path specified. Models will be saved in: "AutogluonModels\ag-20250720_164511"
Beginning AutoGluon training... Time limit = 7200s
AutoGluon will save models to 'c:\Users\TOMAS\Downloads\AutogluonModels\ag-20250720_164511'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.12.2
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          8
GPU Count:          0
Memory Avail:       1.13 GB / 7.89 GB (14.3%)
Disk Space Avail:   82.57 GB / 237.24 GB (34.8%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WAPE,
 'freq': 'MS',
 'hyperparameters': {'DeepAR': {}, 'ETS': {}, 'Naive': {}},
 'known_covariates_names': [],
 'num_val_windows': 5,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'tn',
 'time_limit': 7200,
 'verbosity

In [66]:

# 4. Predecir ene-feb 2020 y revertir log1p
forecast = predictor_advanced.predict(ts_final_train)
forecast_df = forecast['mean'].reset_index()
forecast_df['forecast'] = np.expm1(forecast_df['mean'])

# 5. Filtrar febrero 2020
forecast_feb2020 = forecast_df[forecast_df['timestamp'] == '2019-12-01']
forecast_feb2020 = forecast_feb2020.rename(columns={'item_id': 'product_id'})

# 6. Exportar resultados
forecast_feb2020[['product_id', 'forecast']].rename(columns={'forecast': 'tn'}) \
    .to_csv('predicciones_febrero2020_autogluon_avanzado.csv', index=False)

data with frequency 'IRREG' has been resampled to frequency 'MS'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [68]:
df_agg_filtered

,product_id,periodo,tn
0,20001,201701,934.77222
1,20001,201702,798.01620
2,20001,201703,1303.35771
3,20001,201704,1069.96130
4,20001,201705,1502.20132
...,...,...,...
31206,21276,201908,0.01265
31207,21276,201909,0.01856
31208,21276,201910,0.02079
31209,21276,201911,0.03341


In [ ]:
# Calcular el promedio de los últimos 12 meses para cada product_id en df_agg_filtered
df_agg_filtered_sorted = df_agg_filtered.sort_values(['product_id', 'periodo'])
df_agg_filtered['tn_avg_12m'] = (
    df_agg_filtered_sorted.groupby('product_id')['tn']
    .transform(lambda x: x.rolling(window=12, min_periods=1).mean())
)
df_agg_filtered[['product_id', 'periodo', 'tn', 'tn_avg_12m']]

C:\Users\TOMAS\AppData\Local\Temp\ipykernel_13468\1691242631.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agg_filtered['tn_avg_12m'] = (


,product_id,periodo,tn,tn_avg_12m
0,20001,201701,934.77222,934.772220
1,20001,201702,798.01620,866.394210
2,20001,201703,1303.35771,1012.048710
3,20001,201704,1069.96130,1026.526858
4,20001,201705,1502.20132,1121.661750
...,...,...,...,...
31206,21276,201908,0.01265,0.062132
31207,21276,201909,0.01856,0.055907
31208,21276,201910,0.02079,0.051518
31209,21276,201911,0.03341,0.049506


In [72]:

df_agg_filtered

,product_id,periodo,tn,tn_avg_12m
0,20001,201701,934.77222,934.772220
1,20001,201702,798.01620,866.394210
2,20001,201703,1303.35771,1012.048710
3,20001,201704,1069.96130,1026.526858
4,20001,201705,1502.20132,1121.661750
...,...,...,...,...
31206,21276,201908,0.01265,0.062132
31207,21276,201909,0.01856,0.055907
31208,21276,201910,0.02079,0.051518
31209,21276,201911,0.03341,0.049506


In [75]:
# 1. Asegurar tipo de dato
df_agg_filtered['periodo'] = df_agg_filtered['periodo'].astype(int)

# 2. Filtrar cada rango
df_A = df_agg_filtered[(df_agg_filtered['periodo'] >= 201811) & (df_agg_filtered['periodo'] <= 201910)]
df_B = df_agg_filtered[(df_agg_filtered['periodo'] >= 201901) & (df_agg_filtered['periodo'] <= 201912)]

# 3. Agrupar y calcular promedio mensual por producto
prom_A = df_A.groupby('product_id')['tn'].mean().reset_index().rename(columns={'tn': 'promedio_201711_201810'})
prom_B = df_B.groupby('product_id')['tn'].mean().reset_index().rename(columns={'tn': 'promedio_201901_201912'})

# 4. Unir ambas tablas
promedios1 = pd.merge(prom_A, prom_B, on='product_id', how='outer')

# 5. Exportar a Excel
promedios1.to_excel('promedios_mensuales_productos.xlsx', index=False)


C:\Users\TOMAS\AppData\Local\Temp\ipykernel_13468\3472274916.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agg_filtered['periodo'] = df_agg_filtered['periodo'].astype(int)


In [ ]:
# Crear tabla pivote: filas=product_id, columnas=periodo (desde 201811), valores=tn
tabla_pivot = df_agg_filtered[df_agg_filtered['periodo'] >= 201811].pivot(
    index='product_id',
    columns='periodo',
    values='tn'
)


periodo,201811,201812,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912
product_id,,,,,,,,,,,,,,
20001,1813.01511,1486.68669,1275.77351,1259.09363,1470.65653,1647.63848,1629.78233,1109.93769,1678.99318,1261.34529,1660.00561,1561.50552,1397.37231,1504.68856
20002,1766.81068,1009.45458,1266.78751,1043.01349,1083.62552,1287.62346,1034.98927,928.36431,1066.44999,813.78215,1090.18771,1979.53635,1423.57739,1087.30855
20003,1206.91773,769.82869,964.76919,758.32657,638.04010,565.33774,590.12515,662.38654,715.20314,635.59563,967.77116,1081.36645,948.29393,892.50129
20004,802.34669,585.56477,511.33713,441.70332,619.77084,466.70901,603.31081,667.19411,521.71519,482.13372,786.17140,1064.69633,723.94206,637.90002
20005,469.26344,372.63428,363.58438,409.89950,488.21387,624.99880,897.26297,876.39696,745.74978,536.66800,879.52808,996.78275,606.91173,593.24443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21263,0.39115,0.06068,0.04376,0.05927,0.06636,0.01835,0.02258,0.00988,0.03387,0.03388,0.01128,0.01552,0.03247,0.01270
21265,NaN,NaN,NaN,NaN,0.01593,0.36405,0.17635,0.05121,0.02959,0.01593,0.01707,0.10921,0.06600,0.05007
21266,NaN,NaN,NaN,NaN,0.01707,0.36178,0.17634,0.05235,0.05916,0.01480,0.02844,0.11831,0.06713,0.05121


In [77]:
tabla_pivot.to_excel('tabla_pivot_productos.xlsx')